In [1]:
#importing neccessary libraries
import requests
import bs4

#storing assignment url
url = "https://www.linkedin.com/jobs/search/?currentJobId=3343624705&geoId=107025191&keywords=data%20scientist%2C%20barcelona&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true&position=4&pageNum=0"

#retrieving html tree
source = requests.get(url)

#parsing html tree
soup = bs4.BeautifulSoup(source.text , "html.parser")


In [2]:
#creating list of neccessary tags
info_tag = soup.find_all("div", class_ = "base-search-card__info")
url_tags = soup.find_all("a", class_ = "base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")

In [6]:
#defining all neccessary lists
job_titles = []
company_names = []
locations = []
states = []
posting_dates = []
offer_urls = []
applicants_all = []
senority_levels = []
employment_types = []
industries = []
python_required_list = []
application_through_linkedIn_list = []

#looping through individual tags
for x,y in zip(info_tag, url_tags):
    
    ##
    #extracting job title and adding to list
    job_title = x.h3.text.strip()
    job_titles.append(job_title)
    
    
    ##
    #extracting company name and adding to list
    company_name = x.find("a", class_ = "hidden-nested-link").text.strip()
    company_names.append(company_name)
    
    
    ##
    #extracting location information and adding to list
    location = x.find("span", class_ = "job-search-card__location").text.strip()
    locations.append(location)
    
    
    ##
    #extracting and classifying states and adding to list
    if x.find("span", class_ ="result-benefits__text") == None:
        state = None
    elif x.find("span", class_ ="result-benefits__text").text.strip() == "Actively Hiring":
        state = "On-going"
    elif x.find("span", class_ ="result-benefits__text").text.strip() == "Be an early applicant":
        state = "Early applicant"
    else:
        state = "other"
    states.append(state)
    
    
    ##
    #extracting posting date, assigning none if nothing is mentioned then adding to list
    if x.find("time", class_ ="job-search-card__listdate") == None:
        posting_date = None
    else:
        posting_date = x.find("time", class_ ="job-search-card__listdate").text.strip()
    posting_dates.append(posting_date)
    
    offer_url = y["href"]
    offer_urls.append(offer_url)
    
    
    
    ##
    #using 'offer url' to extract html tree from another detail job listing web page
    source2 = requests.get(offer_url)
    soup2 = bs4.BeautifulSoup(source2.text, "html.parser") 
    
    
    #creating a loop to make sure the site loads through brute force
    while soup2.find("h4", class_ = "top-card-layout__second-subline font-sans text-sm leading-open text-color-text-low-emphasis mt-0.5") == None:
        source2 = requests.get(offer_url)
        soup2 = bs4.BeautifulSoup(source2.text, "html.parser")
        
    
    #extracting applications data then using position of 'applicants' word to take out number of applicants and adding to list
    yo = soup2.find("h4", class_ = "top-card-layout__second-subline font-sans text-sm leading-open text-color-text-low-emphasis mt-0.5").text.strip().split()
    if 'solicitudes' in yo:
        index_num = yo.index("solicitudes")
    elif 'solicitantes' in yo:
        index_num = yo.index('solicitantes')
        
    applicants = int(yo[index_num - 1])
    applicants_all.append(applicants)
    
    
    
    
    
    ##
    #creating lists of headings of "Senority Level","Employment Type" and "Industry"
    if  soup2.find_all("h3", class_ = "description__job-criteria-subheader") == None:
        yo = None
    else:
        yo = soup2.find_all("h3", class_ = "description__job-criteria-subheader")
    heading_list = []
    for x in yo:
        heading = x.text.strip()
        heading_list.append(heading)
        
    
    #checking if the headings exist in the list and if they do extracting their index numbers and finally using those index numbers to extract the respective data within them
    if "Nivel de antigüedad" in heading_list:
        precious = heading_list.index("Nivel de antigüedad")
        senority_level = soup2.find_all("span", class_ = "description__job-criteria-text description__job-criteria-text--criteria")[precious].text.strip()
        
    else:
        senority_level = None
    
    
    if "Tipo de empleo" in heading_list:
        precious = heading_list.index("Tipo de empleo")
        employment_type = soup2.find_all("span", class_ = "description__job-criteria-text description__job-criteria-text--criteria")[precious].text.strip()
        
    else:
        employment_type = None
        
    
    if "Sectores" in heading_list:
        precious = heading_list.index("Sectores")
        industry = soup2.find_all("span", class_ = "description__job-criteria-text description__job-criteria-text--criteria")[precious].text.strip()
        
    else:
        industry = None
    
    #appending them all into their respective lists
    senority_levels.append(senority_level)
    employment_types.append(employment_type)
    industries.append(industry)
    
    
    
    
    
    ##
    #extracting job description through web page and then using case conversion on the description to convert it to lowercase and checking for existence of the word "python"
    if  soup2.find("div", class_ = "show-more-less-html__markup") == None:
        yo = None
    else:
        yo = soup2.find("div", class_ = "show-more-less-html__markup").text.lower()

    
    
    if "python" in yo:
        python_required = True
    else:
        python_required = False
        
    #appending it to a list
    python_required_list.append(python_required)
    
    
    
    
    
    ##
    #checking if "Apply" button type exists and classifying accordingly
    if  soup2.find("button", class_ = "apply-button apply-button--default top-card-layout__cta mt-2 ml-1.5 h-auto babybear:flex-auto top-card-layout__cta--primary btn-md btn-primary") == None:
        application_through_linkedIn = False
    
    else:
        application_through_linkedIn = True
        
        
    #appending it to a list 
    application_through_linkedIn_list.append(application_through_linkedIn)
 

In [4]:
#importing pandas and numpy
import pandas as pd
import numpy as np

#creating a dictionary of all lists
dict_jobs = {"Company Name" : company_names,
             "Job Title" : job_titles,
             "Location" : locations,
             "State" : states, 
             "Posting Date" : posting_dates,
             "Offer URL" : offer_urls,
             "Number of Applicants" : applicants_all,
             "Seniority" : senority_levels,
             "Employment Type" : employment_types,
             "Industry" : industries,
             "Python Required" : python_required_list,
             "Application through Linkedin" : application_through_linkedIn_list}


#creating a dataframe from the dictionary
df_jobs = pd.DataFrame(dict_jobs)

#replacing all None values with numpy NaN
df_jobs = df_jobs.fillna(value=np.nan)

In [5]:
df_jobs

,Company Name,Job Title,Location,State,Posting Date,Offer URL,Number of Applicants,Seniority,Employment Type,Industry,Python Required,Application through Linkedin
0,Babel,Data Science,Greater Barcelona Metropolitan Area,On-going,3 weeks ago,https://es.linkedin.com/jobs/view/data-science...,129,Algo de responsabilidad,Jornada completa,Servicios de TI y consultoría de TI,True,True
1,TFS HealthScience - Contract Research Organiza...,Beca Data Scientist Associate (Barcelona),"Barcelona, Catalonia, Spain",On-going,2 weeks ago,https://es.linkedin.com/jobs/view/beca-data-sc...,59,Sin experiencia,Jornada completa,Fabricación de productos farmacéuticos,False,True
2,PepsiCo,Data Scientist,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://es.linkedin.com/jobs/view/data-scienti...,83,Intermedio,Jornada completa,Servicios de alimentos y bebidas y Manufactura,True,False
3,Fundació de Recerca Sant Joan de Déu,Data Scientist,"Esplugues de Llobregat, Catalonia, Spain",NaN,1 week ago,https://es.linkedin.com/jobs/view/data-scienti...,37,No corresponde,Jornada completa,Servicios de bienestar y ejercicio,True,False
4,Gauss & Neumann,Data Scientist (Andorra),"Barcelona, Catalonia, Spain",On-going,1 month ago,https://es.linkedin.com/jobs/view/data-scienti...,200,NaN,Jornada completa,NaN,True,True
5,Product Madness 📱🎮,Data Scientist,"Barcelona, Catalonia, Spain",On-going,2 weeks ago,https://es.linkedin.com/jobs/view/data-scienti...,122,Sin experiencia,Jornada completa,Videojuegos,True,False
6,MANGO,DATA SCIENTIST INTEN,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://es.linkedin.com/jobs/view/data-scienti...,35,Sin experiencia,Jornada completa,Industria textil y moda para la venta minorista,True,False
7,MANGO,Insights Data Scientist,"Barcelona, Catalonia, Spain",On-going,1 month ago,https://es.linkedin.com/jobs/view/insights-dat...,28,Sin experiencia,Jornada completa,Industria textil y moda para la venta minorista,True,False
8,PepsiCo,Data Scientist,"Barcelona, Catalonia, Spain",On-going,3 days ago,https://es.linkedin.com/jobs/view/data-scienti...,27,Intermedio,Jornada completa,Servicios de alimentos y bebidas y Manufactura,True,False
9,PepsiCo,Data Scientist,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://es.linkedin.com/jobs/view/data-scienti...,43,Intermedio,Jornada completa,Servicios de alimentos y bebidas y Manufactura,True,False


In [156]:
df_jobs.dtypes

Company Name                    object
Job Title                       object
Location                        object
State                           object
Posting Date                    object
Offer URL                       object
Number of Applicants             int64
Seniority                       object
Employment Type                 object
Industry                        object
Python Required                   bool
Application through Linkedin      bool
dtype: object